In [1]:
import numpy as np
import open3d as o3d
import os
from makegripper_points import  plot_gripper_pro_max
from scipy.spatial.transform import Rotation as R

# Include your original plot_gripper_pro_max function here or import it if it's from another module

def visualize_selected_grasps(data_dir, grasp_indices):
    # Load saved data
    cloud_path = os.path.join(data_dir, "cloud.ply")
    grasps_path = os.path.join(data_dir, "grasps.npy")
    
    if not os.path.exists(cloud_path) or not os.path.exists(grasps_path):
        print("Error: cloud.ply or grasps.npy not found in the directory.")
        return

    cloud = o3d.io.read_point_cloud(cloud_path)
    grasp_data = np.load(grasps_path, allow_pickle=True).item()

    translations = grasp_data['translations']
    rotations = grasp_data['rotations']
    widths = grasp_data['widths']
    heights = grasp_data['heights']
    scores = grasp_data['scores']
    gripper_points = grasp_data['gripper_points']

    pcds = []
    dots = []

    for idx in grasp_indices:
        if idx >= len(translations):
            print(f"Index {idx} out of range.")
            continue

        t = translations[idx]
        R_mat = rotations[idx]
        width = widths[idx]
        height = heights[idx]
        score = scores[idx]
        gripper_point = gripper_points[idx]

        # Gripper mesh
        gripper_pcd, _ = plot_gripper_pro_max(t, R_mat, width=width, depth=height, score=score, color=[0, 1, 0])
        pcds.append(gripper_pcd)

        # Red dots
        for p in gripper_point:
            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.002)
            sphere.translate(p)
            sphere.paint_uniform_color([1, 0, 0])
            dots.append(sphere)


        rpy = R.from_matrix(R_mat).as_euler('xyz', degrees=True) 

        print(f"\nGrasp Index: {idx}")
        print(f"  Score: {score:.4f}")
        print(f"  Translation (x, y, z): {t*1000}")
        print(f"  Rotation (roll, pitch, yaw) [deg]: {rpy}")
        print(f"  Width: {width:.4f}  | Depth: {height:.4f}")


    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
    o3d.visualization.draw_geometries([cloud, axis] + pcds + dots)




visualize_selected_grasps('./real_life_output/real_life/test1', grasp_indices=[0,1])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.

Grasp Index: 0
  Score: 0.3296
  Translation (x, y, z): [556.19852157  96.84616325 121.85274518]
  Rotation (roll, pitch, yaw) [deg]: [-101.89943203   84.98887378  140.48428298]
  Width: 0.0658  | Depth: 0.0300

Grasp Index: 1
  Score: 0.3191
  Translation (x, y, z): [537.38266043 102.67822148  25.65250945]
  Rotation (roll, pitch, yaw) [deg]: [-163.7599652    31.45111252   -3.53959523]
  Width: 0.0943  | Depth: 0.0300
